In [273]:
commands = {"STOP": "FFFF", "MOV": "1", "ADD": "2", "OUT": "3", "ADDN": "4", "SET": "5", "EQ": "6", "PUSH": "7", "START": "8", "BEGINFUNC": "9", "ENDFUNC": "C"}
base = 16

In [274]:
for i in commands:
    temp = bin(int(commands[i], 16))[2:]
    commands[i] = '0'*(base - len(temp)) + temp
print(commands)

{'STOP': '1111111111111111', 'MOV': '0000000000000001', 'ADD': '0000000000000010', 'OUT': '0000000000000011', 'ADDN': '0000000000000100', 'SET': '0000000000000101', 'EQ': '0000000000000110', 'PUSH': '0000000000000111', 'START': '0000000000001000', 'BEGINFUNC': '0000000000001001', 'ENDFUNC': '0000000000001100'}


### Обычная архитектура без воможности использования функций

In [262]:
import numpy as np

class VMachine:
    
    def LoadProgram(self, string):
        self.memory = []
        while(len(string) > 0):
            self.memory.append(string[:base])
            string = string[base:]
        #print(self.memory)
    
    def Run(self):
        ptr_next_block = self.memory[3]
        ptr_stack = self.memory[7]
        while self.memory[int(ptr_next_block,2)] != commands["STOP"]:
            #print(int(ptr_next_block,2))
            curr_command = self.memory[int(ptr_next_block,2)]
            ptr_curr_first = self.memory[int(ptr_next_block,2) + 1]
            ptr_curr_second = self.memory[int(ptr_next_block,2) + 2]
            ptr_curr_third = self.memory[int(ptr_next_block,2) + 3]
            
            if curr_command == commands["MOV"]:
                self.memory[int(ptr_curr_first, 2) : int(ptr_curr_first, 2) + 4] = self.memory[int(ptr_curr_third, 2) : int(ptr_curr_third, 2) + 4]
                
            elif curr_command == commands["OUT"]:
                print(int(self.memory[int(ptr_curr_third, 2)] + self.memory[int(ptr_curr_third, 2) + 1] + 
                          self.memory[int(ptr_curr_third, 2) + 2] + self.memory[int(ptr_curr_third, 2) + 3], 2))
                
            elif curr_command == commands["ADD"]:
                temp = int(self.memory[int(ptr_curr_first, 2)] + self.memory[int(ptr_curr_first, 2) + 1] +
                           self.memory[int(ptr_curr_first, 2) + 2] + self.memory[int(ptr_curr_first, 2) + 3],2) + int(self.memory[int(ptr_curr_third, 2)] +
                                                                                                                      self.memory[int(ptr_curr_third, 2) + 1] +
                                                                                                                      self.memory[int(ptr_curr_third, 2) + 2] +
                                                                                                                      self.memory[int(ptr_curr_third, 2) + 3],2)

                temp = bin(temp)[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                self.memory[int(ptr_curr_first, 2)] = temp[:base]
                self.memory[int(ptr_curr_first, 2) + 1] = temp[base:base*2]
                self.memory[int(ptr_curr_first, 2) + 2] = temp[base*2:base*3]
                self.memory[int(ptr_curr_first, 2) + 3] = temp[base*3:base*4]
                
            elif curr_command == commands["ADDN"]:
                temp = int(self.memory[int(ptr_curr_first, 2)] + self.memory[int(ptr_curr_first, 2) + 1] +
                           self.memory[int(ptr_curr_first, 2) + 2] + self.memory[int(ptr_curr_first, 2) + 3],2) + int(ptr_curr_third, 2)
                temp = bin(temp)[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                self.memory[int(ptr_curr_first, 2)] = temp[:base]
                self.memory[int(ptr_curr_first, 2) + 1] = temp[base:base*2]
                self.memory[int(ptr_curr_first, 2) + 2] = temp[base*2:base*3]
                self.memory[int(ptr_curr_first, 2) + 3] = temp[base*3:base*4]
                
            elif curr_command == commands["SET"]:
                self.memory[int(ptr_curr_first, 2)] = '0'*base
                self.memory[int(ptr_curr_first, 2) + 1] = '0'*base
                self.memory[int(ptr_curr_first, 2) + 2] = '0'*base
                self.memory[int(ptr_curr_first, 2) + 3] = ptr_curr_third
            
            elif curr_command == commands["EQ"]:
                temp1 = self.memory[int(ptr_curr_first, 2)] + self.memory[int(ptr_curr_first, 2) + 1] + self.memory[int(ptr_curr_first, 2) + 2] + self.memory[int(ptr_curr_first, 2) + 3]
                temp2 = self.memory[int(ptr_curr_second, 2)] + self.memory[int(ptr_curr_second, 2) + 1] + self.memory[int(ptr_curr_second, 2) + 2] + self.memory[int(ptr_curr_second, 2) + 3]
                if temp1 != temp2:
                    temp = bin(int(self.memory[3], 2) + 4*int(ptr_curr_third, 2))[2:]
                    temp = '0'*(base - len(temp)) + temp
                    self.memory[3] = temp
            
            temp = bin(int(self.memory[3], 2) + 4)[2:]
            temp = '0'*(base - len(temp)) + temp
            self.memory[3] = temp
            ptr_next_block = self.memory[3]
               
        
def accembler(fin, fout):
    var = dict([])
    func = dict([])
    curr_ptr = 0
    for line in fin:
        words = line.split()
        if (len(words) >=2 and words[1] == "="):
            temp = bin(curr_ptr)[2:]
            var[words[0]] = '0'*(base - len(temp)) + temp
            num = bin(int(words[2], 16))[2:]
            fout.write('0'*(base*4 - len(num)) + num)
        else:
            i1 = words[0]
            strm = ""
            if i1 == "OUT":
                strm += commands[i1]
                strm += '0'*2*base
                strm += var[words[1]]
            elif i1 == "MOV":
                strm += commands[i1]
                strm += var[words[1]]
                strm += '0'*base
                strm += var[words[2]]
            elif i1 == "ADD":
                strm += commands[i1]
                strm += var[words[1]]
                strm += '0'*base
                strm += var[words[2]]
            elif i1 == "ADDN":
                strm += commands[i1]
                strm += var[words[1]]
                strm += '0'*base
                temp = bin(int(words[2]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "SET":
                strm += commands[i1]
                strm += var[words[1]]
                strm += '0'*base
                temp = bin(int(words[2]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "EQ":
                strm += commands[i1]
                strm += var[words[1]]
                strm += var[words[2]]
                temp = bin(int(words[3]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "STOP":
                strm += commands[i1]
                strm += '0'*3*base
            fout.write(strm)
        
        curr_ptr += 4

In [263]:
fin = open("input.txt", 'r')
fout = open("output.txt", 'w')
accembler(fin, fout)
fin.close()
fout.close()

In [264]:
fin = open("output.txt", 'r')

machine = VMachine()
machine.LoadProgram(fin.readline())

In [265]:
machine.Run()
fin.close()

0
4


### Архитектура с функциями (не работает)

In [283]:
import numpy as np

class VMachine:
    
    def LoadProgram(self, string):
        self.memory = []
        while(len(string) > 0):
            self.memory.append(string[:base])
            string = string[base:]
        #print(self.memory)
    
    def Run(self):
        ptr_next_block = self.memory[3]
        ptr_stack = self.memory[7]
        while self.memory[int(ptr_next_block,2)] != commands["STOP"]:
            #print(int(ptr_next_block,2))
            curr_command = self.memory[int(ptr_next_block,2)]
            ptr_curr_first = self.memory[int(ptr_next_block,2) + 1]
            ptr_curr_second = self.memory[int(ptr_next_block,2) + 2]
            ptr_curr_third = self.memory[int(ptr_next_block,2) + 3]
            
            if curr_command == commands["MOV"]:
                if int(self.memory[11],2) != int(self.memory[15], 2):
                    ptr_curr_first = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_first,2))*4 - 1]
                    ptr_curr_third = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_third,2))*4 - 1]
                self.memory[int(ptr_curr_first, 2) : int(ptr_curr_first, 2) + 4] = self.memory[int(ptr_curr_third, 2) : int(ptr_curr_third, 2) + 4]
                
            elif curr_command == commands["OUT"]:
                print(int(self.memory[int(ptr_curr_third, 2)] + self.memory[int(ptr_curr_third, 2) + 1] + 
                          self.memory[int(ptr_curr_third, 2) + 2] + self.memory[int(ptr_curr_third, 2) + 3], 2))
                
            elif curr_command == commands["ADD"]:
                if int(self.memory[11],2) != int(self.memory[15], 2):
                    ptr_curr_first = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_first,2))*4 - 1]
                    ptr_curr_third = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_third,2))*4 - 1]
                temp = int(self.memory[int(ptr_curr_first, 2)] + self.memory[int(ptr_curr_first, 2) + 1] +
                           self.memory[int(ptr_curr_first, 2) + 2] + self.memory[int(ptr_curr_first, 2) + 3],2) + int(self.memory[int(ptr_curr_third, 2)] +
                                                                                                                      self.memory[int(ptr_curr_third, 2) + 1] +
                                                                                                                      self.memory[int(ptr_curr_third, 2) + 2] +
                                                                                                                      self.memory[int(ptr_curr_third, 2) + 3],2)

                temp = bin(temp)[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                self.memory[int(ptr_curr_first, 2)] = temp[:base]
                self.memory[int(ptr_curr_first, 2) + 1] = temp[base:base*2]
                self.memory[int(ptr_curr_first, 2) + 2] = temp[base*2:base*3]
                self.memory[int(ptr_curr_first, 2) + 3] = temp[base*3:base*4]
                
            elif curr_command == commands["ADDN"]:
                if int(self.memory[11],2) != int(self.memory[15], 2):
                    ptr_curr_first = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_first,2))*4 - 1]
                temp = int(self.memory[int(ptr_curr_first, 2)] + self.memory[int(ptr_curr_first, 2) + 1] +
                           self.memory[int(ptr_curr_first, 2) + 2] + self.memory[int(ptr_curr_first, 2) + 3],2) + int(ptr_curr_third, 2)
                temp = bin(temp)[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                self.memory[int(ptr_curr_first, 2)] = temp[:base]
                self.memory[int(ptr_curr_first, 2) + 1] = temp[base:base*2]
                self.memory[int(ptr_curr_first, 2) + 2] = temp[base*2:base*3]
                self.memory[int(ptr_curr_first, 2) + 3] = temp[base*3:base*4]
                
            elif curr_command == commands["SET"]:
                if int(self.memory[11],2) != int(self.memory[15], 2):
                    ptr_curr_first = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_first,2))*4 - 1]
                self.memory[int(ptr_curr_first, 2)] = '0'*base
                self.memory[int(ptr_curr_first, 2) + 1] = '0'*base
                self.memory[int(ptr_curr_first, 2) + 2] = '0'*base
                self.memory[int(ptr_curr_first, 2) + 3] = ptr_curr_third
            
            elif curr_command == commands["EQ"]:
                if int(self.memory[11],2) != int(self.memory[15], 2):
                    ptr_curr_second = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_second,2))*4 - 1]
                    ptr_curr_first = self.memory[int(self.memory[11], 2) - (int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_first,2))*4 - 1]
                temp1 = self.memory[int(ptr_curr_first, 2)] + self.memory[int(ptr_curr_first, 2) + 1] + self.memory[int(ptr_curr_first, 2) + 2] + self.memory[int(ptr_curr_first, 2) + 3]
                temp2 = self.memory[int(ptr_curr_second, 2)] + self.memory[int(ptr_curr_second, 2) + 1] + self.memory[int(ptr_curr_second, 2) + 2] + self.memory[int(ptr_curr_second, 2) + 3]
                if temp1 != temp2:
                    temp = bin(int(self.memory[3], 2) + 4*int(ptr_curr_third, 2))[2:]
                    temp = '0'*(base - len(temp)) + temp
                    self.memory[3] = temp
            
            elif curr_command == commands["PUSH"]:
                ptr_stack = self.memory[7]
                self.memory[int(ptr_stack, 2)] = '0'*base
                self.memory[int(ptr_stack, 2) + 1] = '0'*base
                self.memory[int(ptr_stack, 2) + 2] = '0'*base
                if int(ptr_curr_second,2) != 0:
                    self.memory[int(ptr_stack, 2) + 3] = self.memory[(int(self.memory[int(self.memory[11], 2) + 2], 2) - int(ptr_curr_second,2))*4 - 1]
                else:
                    self.memory[int(ptr_stack, 2) + 3] = ptr_curr_third
                temp = bin(int(self.memory[7], 2) + 4)[2:]
                temp = '0'*(base - len(temp)) + temp
                self.memory[7] = temp
                    
            elif curr_command == commands["START"]:
                self.memory[11] = self.memory[7]
                self.memory[int(self.memory[7], 2)] = self.memory[3]
                self.memory[int(self.memory[7], 2) + 1] = ptr_curr_first
                self.memory[int(self.memory[7], 2) + 2] = ptr_curr_third
                self.memory[int(self.memory[7], 2) + 3] = '0'*base
                self.memory[3] = ptr_curr_first
                temp = bin(int(self.memory[7], 2) + 4)[2:]
                temp = '0'*(base - len(temp)) + temp
                self.memory[7] = temp
            
            elif curr_command == commands["ENDFUNC"]:
                top = int(self.memory[7], 2) - 4
                nnn = int(self.memory[top + 2], 2)
                self.memory[3] = self.memory[top]
                temp = bin(top - (nnn+1)*4)[2:]
                temp = '0'*(base - len(temp)) + temp
                self.memory[7] = temp
                self.memory[11] = temp
            
            temp = bin(int(self.memory[3], 2) + 4)[2:]
            temp = '0'*(base - len(temp)) + temp
            self.memory[3] = temp
            ptr_next_block = self.memory[3]
               
        
def accembler(fin, fout):
    var = dict([])
    func = dict([])
    curr_ptr = 0
    for line in fin:
        words = line.split()
        if (len(words) >=2 and words[1] == "="):
            temp = bin(curr_ptr)[2:]
            var[words[0]] = '0'*(base - len(temp)) + temp
            num = bin(int(words[2], 16))[2:]
            fout.write('0'*(base*4 - len(num)) + num)
        else:
            i1 = words[0]
            strm = ""
            if i1 == "OUT":
                strm += commands[i1]
                strm += '0'*2*base
                strm += var[words[1]]
            elif i1 == "MOV":
                strm += commands[i1]
                if words[1].isdigit():
                    temp = bin(int(words[1]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                    strm += '0'*base
                    temp = bin(int(words[2]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                else:
                    strm += var[words[1]]
                    strm += '0'*base
                    strm += var[words[2]]
            elif i1 == "ADD":
                strm += commands[i1]
                if words[1].isdigit():
                    temp = bin(int(words[1]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                    strm += '0'*base
                    temp = bin(int(words[2]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                else:
                    strm += var[words[1]]
                    strm += '0'*base
                    strm += var[words[2]]
            elif i1 == "ADDN":
                strm += commands[i1]
                if words[1].isdigit():
                    temp = bin(int(words[1]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                else:
                    strm += var[words[1]]
                strm += '0'*base
                temp = bin(int(words[2]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "SET":
                strm += commands[i1]
                if words[1].isdigit():
                    temp = bin(int(words[1]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                else:
                    strm += var[words[1]]
                strm += '0'*base
                temp = bin(int(words[2]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "EQ":
                strm += commands[i1]
                if words[1].isdigit():
                    temp = bin(int(words[1]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                    temp = bin(int(words[2]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                else:
                    strm += var[words[1]]
                    strm += var[words[2]]
                temp = bin(int(words[3]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "PUSH":
                strm += commands[i1]
                strm += '0'*base
                if words[1].isdigit():
                    temp = bin(int(words[1]))[2:]
                    temp = '0'*(base - len(temp)) + temp[-base:]
                    strm += temp
                    strm += '0'*base
                else:
                    strm += '0'*base
                    strm += var[words[1]]
            elif i1 == "BEGINFUNC":
                strm += commands[i1]
                strm += '0'*base*3
                temp = bin(curr_ptr)[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                func[words[1]] = temp
            elif i1 == "ENDFUNC":
                strm += commands[i1]
                strm += '0'*base*2
                strm += func[words[1]]
            elif i1 == "START":
                strm += commands[i1]
                strm += func[words[1]]
                strm += '0'*base
                temp = bin(int(words[2]))[2:]
                temp = '0'*(base - len(temp)) + temp[-base:]
                strm += temp
            elif i1 == "STOP":
                strm += commands[i1]
                strm += '0'*3*base
            fout.write(strm)
        
        curr_ptr += 4
    fout.write('0'*50*4*16)

In [287]:
fin = open("input.txt", 'r')
fout = open("output.txt", 'w')
accembler(fin, fout)
fin.close()
fout.close()

In [288]:
fin = open("output.txt", 'r')

machine = VMachine()
machine.LoadProgram(fin.readline())

In [289]:
machine.Run()
fin.close()

2


In [ ]:
def accembler(fin, fout):
    for line in fin:
        words = line.split()
        if words[0] == "STOP":
            
        elif words[0] == "MOV":
            
        elif words[0] == "ADD":
            
        elif words[0] == "EQ":
        
        i1 = commands[words[0]]
        i1 = bin(int(i1, 16))[2:]
        fout.write('0'*(8 - len(i1)) + i1)
        for i in range(1, len(words)):
            temp = bin(int(words[i], 16))[2:]
            fout.write(" "+'0'*(8 - len(temp)) + temp)
        fout.write('\n')
        
        
        '''
            elif curr_command == bin(int(commands["ADD"], 16)):
                new_first = bin(int(string[int(ptr_curr_first, 2)*base, int(ptr_curr_first, 2)*base + 4*base], 2) + 
                int(string[int(ptr_curr_third, 2)*base, int(ptr_curr_third, 2)*base + 4*base], 2))    #summing
                
                new_first = new_first[-base:]
                
                string[int(ptr_curr_first, 2)*base, int(ptr_curr_first, 2)*base + 4*base] = 
                '0'*(4*base - len(new_first)) + new_first 
                
            elif curr_command == bin(int(commands["ADDN"], 16)):
                new_first = bin(int(string[int(ptr_curr_first, 2)*base, int(ptr_curr_first, 2)*base + 4*base], 2) + 
                                int(ptr_curr_second+ptr_curr_third, 2))
                
                new_first = new_first[-base:]
                
                string[int(ptr_curr_first, 2)*base, int(ptr_curr_first, 2)*base + 4*base] = 
                '0'*(4*base - len(new_first)) + new_first 
                
            elif curr_command == bin(int(commands["SET"], 16)):
                new_first = ptr_curr_second + ptr_curr_third
                
                string[int(ptr_curr_first, 2)*base, int(ptr_curr_first, 2)*base + 4*base] = 
                '0'*(4*base - len(new_first)) + new_first 
                
            '''